In [2]:
import numpy as np

In [33]:
entradas = np.array([[0.05, 0.1]])
entradas

array([[ 0.05,  0.1 ]])

In [34]:
entradas.shape

(1, 2)

In [35]:
entradas = np.c_[np.ones(entradas.shape[0]), entradas]
#entradas = np.append(1, entradas)
entradas

array([[ 1.  ,  0.05,  0.1 ]])

In [36]:
entradas.shape

(1, 3)

In [37]:
pesos = np.array(
        [[0.35, 0.35],
         [0.15, 0.25],
         [0.20, 0.30]])
pesos

array([[ 0.35,  0.35],
       [ 0.15,  0.25],
       [ 0.2 ,  0.3 ]])

In [38]:
pesos.shape

(3, 2)

In [39]:
np.dot(entradas, pesos)

array([[ 0.3775,  0.3925]])